<a href="https://colab.research.google.com/github/aiaaee/California_Housing/blob/main/CaliforniaHousing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchvision import transforms
from tqdm import tqdm

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [215]:
data = fetch_california_housing()
# In dataset as_frame is false . it means that we need to create Dataset and do data and target combination
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [216]:
print(X_train.shape , y_train.shape)
print(X_test.shape , y_test.shape)

(16512, 8) (16512,)
(4128, 8) (4128,)


In [217]:
X_train

array([[-0.326196  ,  0.34849025, -0.17491646, ...,  0.05137609,
        -1.3728112 ,  1.27258656],
       [-0.03584338,  1.61811813, -0.40283542, ..., -0.11736222,
        -0.87669601,  0.70916212],
       [ 0.14470145, -1.95271028,  0.08821601, ..., -0.03227969,
        -0.46014647, -0.44760309],
       ...,
       [-0.49697313,  0.58654547, -0.60675918, ...,  0.02030568,
        -0.75500738,  0.59946887],
       [ 0.96545045, -1.07984112,  0.40217517, ...,  0.00707608,
         0.90651045, -1.18553953],
       [-0.68544764,  1.85617335, -0.85144571, ..., -0.08535429,
         0.99543676, -1.41489815]])

In [218]:
# Create a custom PyTorch dataset

toTensorTransform = transforms.Lambda(lambda x: torch.tensor(x, dtype=torch.float32))
class CaliforniaHousingDataset(Dataset):
    def __init__(self, features, targets, transform=None, target_transform=None):
        self.features = features
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.targets[index]
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y

In [219]:
# Create instances of the custom dataset
train_dataset = CaliforniaHousingDataset(X_train, y_train, transform=toTensorTransform,
                                         target_transform=transforms.Lambda(lambda x: torch.tensor(x, dtype=torch.float32).unsqueeze(0)))

test_dataset = CaliforniaHousingDataset(X_test, y_test, transform=toTensorTransform,
                                        target_transform=transforms.Lambda(lambda x: torch.tensor(x, dtype=torch.float32).unsqueeze(0)))

# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [220]:
for data, target in train_loader:
    print(target.size())
    break

torch.Size([64, 1])


In [330]:
class NeuralNet(nn.Module) :
  def __init__(self , in_features , out_features , *args , **kwargs ):
    super().__init__(*args , **kwargs)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(in_features , 200)
    self.fc2 = nn.Linear(200 , 200)
    self.fc3 = nn.Linear(200 , out_features)
  def forward(self , x ):
    x = self.flatten(x)
    output = self.fc1(x)
    output = nn.ReLU()(output)
    output = self.fc2(x)
    output = nn.ReLU()(output)
    return self.fc3(output)


In [331]:
model = NeuralNet(8 , 1 )
model

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=8, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=1, bias=True)
)